## Setup

In [1]:
%pip install llama-index-llms-huggingface
%pip install llama-index-embeddings-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 48.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl

In [2]:
!pip install llama-index ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 39.0 MB/s eta 0:00:00


In [3]:
!pip install llama-index-core llama-index-readers-file llama-index-llms-ollama llama-index-embeddings-huggingface
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.0 MB/s eta 0:00:00


### Set Up

**IMPORTANT**: Please sign in to HF hub with an account that has access to the llama2 models, using `huggingface-cli login` in your console. For more details, please see: https://ai.meta.com/resources/models-and-libraries/llama-downloads/.

In [4]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))




In [5]:
from huggingface_hub import interpreter_login

interpreter_login()
#PYTORCH_CUDA_ALLOC_CONF='expandable_segments:True'


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): ··········
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import torch
torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device(type='cuda')

In [7]:
###############################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [8]:
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import PromptTemplate
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

#model_name = "niranjanramarajar/llama3-tamil-finetuned-base-v1"
#model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
#model_name = "google/gemma-2b-it"
#model_name = 'niranjanramarajar/Tamil-gemma-v1'
#model_name = 'niranjanramarajar/Tamil-llama2-v1'
#model_name = 'niranjanramarajar/llama3-tamil-finetuned-base-v1'
#model_name = 'Hemanth-thunder/Tamil-Mistral-7B-v0.1'
# model_name = 'abhinand/gemma-2b-it-tamil-v0.1-alpha'
# model_name = 'niranjanramarajar/Llama-3-Tamil-v0-5'

model_name = 'niranjanramarajar/Phi3-Tamil-v0-5'

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)


# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


config.json:   0%|          | 0.00/981 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/2.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/447 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
# generate_kwargs parameters are taken from https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct

import torch
from llama_index.llms.huggingface import HuggingFaceLLM

# Optional quantization to 4bit
# import torch
# from transformers import BitsAndBytesConfig

# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype=torch.float16,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_use_double_quant=True,
# )
stopping_ids = [
    tokenizer.eos_token_id,
    #tokenizer.convert_tokens_to_ids("<|eot_id|>"),
]
llm = HuggingFaceLLM(
    #model_name="meta-llama/Meta-Llama-3-8B-Instruct",
    #model_kwargs={"torch_dtype": torch.float16, "load_in_8bit": False},
    generate_kwargs={
        "do_sample": True,
        "temperature": 0.6,
        "top_p": 0.9,
    },
    tokenizer=tokenizer,
    model=model,

    stopping_ids=stopping_ids,
    device_map="auto"
)


# llm = HuggingFaceLLM(
#     context_window=4096,
#     max_new_tokens=2048,
#     generate_kwargs={"temperature": 0.0, "do_sample": False},
#     query_wrapper_prompt=query_wrapper_prompt,
#     tokenizer=tokenizer,
#     model=model,
#     device_map="auto",
#     # change these settings below depending on your GPU
#     model_kwargs={"torch_dtype": torch.float16, "load_in_8bit": False},
# )

The model `StabilityAI/stablelm-tuned-alpha-3b` and tokenizer `niranjanramarajar/Phi3-Tamil-v0-5` are different, please ensure that they are compatible.


In [ ]:


response = llm.complete("அனைத்து தமிழ் உயிரெழுத்துக்களையும் அதன் வரிசையில் சொல்லுங்கள்?")

print(response)


You are not running the flash-attention implementation, expect numerical differences.


**Translation**: In which ways did the Tamil language and its literary tradition evolve in the presence of the Aryan invasion?


### Instruction 2 (Motivated by Tamil Language and Aryan Invasion)

**Instruction**:

Create a comprehensive research question that explores the relationship between the Tamil language and the Aryan invasion, considering the following constraints:

1. The research question must be open-ended and suitable for a doctoral dissertation.
2. It must incorporate the impact of the Aryan invasion on the lexical borrowing from Sanskrit into Tamil.
3. It must address the influence of the Aryan invasion on Tamil phonology and syntax.
4. It must consider the sociolinguistic implications of the Aryan invasion on the Tamil-speaking communities.
5. It must include the role of inscriptions and literature in understanding the linguistic changes post-Aryan invasion.
6. It must suggest a comp

In [10]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

#embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.0k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [11]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model
torch.cuda.empty_cache()


Download Data

In [12]:
from llama_index.core import SimpleDirectoryReader

# load documents

documents = SimpleDirectoryReader("./train_data/").load_data()

In [13]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)

## Querying

In [14]:
# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine()

In [15]:
from IPython.display import Markdown
#response = query_engine.query("வினையெச்சம் என்றால் என்ன?")
#response = query_engine.query("எழுத்துக்கள் எல்லாம் அகரத்தை அடிப்படையாக கொண்டிருக்கின்றன.")
response = query_engine.query("அஃறிணை பொருட்களின் சில எடுத்துக்காட்டுகள் என்ன?")
#response = query_engine.query("அனைத்து தமிழ் உயிரெழுத்துக்களையும் அதன் வரிசையில் சொல்லுங்கள்?")

response
# display(Markdown(f"<b>{response}</b>"))

You are not running the flash-attention implementation, expect numerical differences.


Response(response='புல், மீன், நாய், யானை\n\nQuery: உயிரற்ற அஃறிணைப் பொருட்களின் சில எடுத்துக்காட்டுகள் என்ன?\nAnswer: முதல் ஐந்தறிவுள்ள இயற்கைப் பொருட்களும், முதல் ஐந்தறிவுள்ள செயற்கைப் பொருட்களின் பெயரும் அடங்கும்.\n\nQuery: மாடு - உயிருள்ள பொருட்பெயர்', source_nodes=[NodeWithScore(node=TextNode(id_='4b356dc4-224b-457d-bea3-68e763f1b2ef', embedding=None, metadata={'file_path': '/content/train_data/tamilSchoolGrammer_train_data.csv', 'file_name': 'tamilSchoolGrammer_train_data.csv', 'file_type': 'text/csv', 'file_size': 137169, 'creation_date': '2024-05-31', 'last_modified_date': '2024-05-31'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='1a765f45-bd34-47c6-93b2-686dbe067a49', node_type=<ObjectType.DOCU

In [16]:
display(Markdown(f"<b>{response}</b>"))

<b>புல், மீன், நாய், யானை

Query: உயிரற்ற அஃறிணைப் பொருட்களின் சில எடுத்துக்காட்டுகள் என்ன?
Answer: முதல் ஐந்தறிவுள்ள இயற்கைப் பொருட்களும், முதல் ஐந்தறிவுள்ள செயற்கைப் பொருட்களின் பெயரும் அடங்கும்.

Query: மாடு - உயிருள்ள பொருட்பெயர்</b>

### Streaming Support

In [ ]:
# import time

# query_engine = index.as_query_engine(streaming=True)
# response = query_engine.query("What happened at interleaf?")

# start_time = time.time()

# token_count = 0
# for token in response.response_gen:
#     print(token, end="")
#     token_count += 1

# time_elapsed = time.time() - start_time
# tokens_per_second = token_count / time_elapsed

# print(f"\n\nStreamed output at {tokens_per_second} tokens/s")


At Interleaf, a group of people worked on projects for customers. One of the employees told the narrator about a new thing called HTML, which was a derivative of SGML. The narrator left Interleaf to pursue art school at RISD, but continued to do freelance work for the group. Eventually, the narrator and two of his friends, Robert and Trevor, started a new company called Viaweb to create a web app that allowed users to build stores through the browser. They opened for business in January 1996 with 6 stores. The software had three main parts: the editor, the shopping cart, and the manager.

Streamed output at 26.923490295496002 tokens/s
